In [ ]:
#! pip install python-terrier

!pip install --upgrade python-terrier

!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 97 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 35.5 MB/s 
     |████████████████████████████████| 69 kB 5.9 MB/s 
     |████████████████████████████████| 255 kB 57.0 MB/s 
     |████████████████████████████████| 41 kB 270 kB/s 
     |████████████████████████████████| 294 kB 54.1 MB/s 
     |████████████████████████████████| 1.8 MB 46.5 MB/s 
     |████████████████████████████████| 6.4 MB 41.8 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 126 kB 61.4 MB/s 
     |████████████████████████████████| 291 kB 68.6 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=21d713a827a8c7f5373ed23e66e5b8d9ec088d88102713020291572e9061b785
  Stored in directory: /root/.cac

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#libraries

import pyterrier as pt
#from pyterrier import IndexingType
import pandas as pd
#import regex as re
import string 
from nltk.stem.porter import *
from nltk.tokenize import WordPunctTokenizer

In [ ]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation+'‐'+'′'+'–'+'‘'+'’'+'“'+'”'])
    return punctuationfree

import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def tokenization(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)
  return tokenized_text

def remove_stopwords(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)
  tokenized_text_without_stopwords = []
  for token in tokenized_text:
    if token.lower() not in stop_words:
      tokenized_text_without_stopwords.append(token)
  return " ".join(tokenized_text_without_stopwords)

def porter_stemmer(text):
  stemmer = PorterStemmer()
  return_text = stemmer.stem(text)

  return return_text

def remove_number_after_space(text):
  text = re.sub('\s(\d)+', ' ', text, regex=True)
  return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#load datasets and init pyterrier

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

root_dir = '/content/gdrive/MyDrive'
base_dir = root_dir + '/Progetto IR/data'
#base_dir = root_dir + "/IR_Dataset"

collection = pd.read_pickle(base_dir + "/collection.pkl")
qrels = pd.read_pickle(base_dir + "/qrels.pkl")
queries = pd.read_pickle(base_dir + "/queries.pkl")


In [ ]:
from gensim import downloader
from gensim.models import Word2Vec
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
wiki_model = downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def expand_query(model, query):
  tokenized_query = WordPunctTokenizer().tokenize(query)
  expanded_query = []
  for word in tokenized_query:
    try:
      ms = model.most_similar(word, topn = 3)
    except KeyError:
      continue
    expanded_query.append(word)
    for response in ms:
      expanded_query.append(response[0])
  return " ".join(expanded_query)

In [ ]:
pt.set_property("termpipelines", "")
pt.set_property("termpipelines", "Stopwords")

pd_indexer = pt.DFIndexer("/pd_index", overwrite = True)
indexref = pd_indexer.index(collection["text"], collection["docno"], collection["date"], collection["title"])


17:06:04.372 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
17:08:20.169 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
17:08:20.274 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 192509
Number of terms: 188603
Number of postings: 11824971
Number of fields: 0
Number of tokens: 16819835
Field names: []
Positions:   false



In [ ]:
display(queries.head())

queries['query_adhoc'] = [el.lower() for el in queries['query_adhoc']]
queries['query_adhoc'] = [remove_punctuation(el) for el in queries['query_adhoc']]
#queries['query_adhoc'] = [remove_stopwords(el) for el in queries['query_adhoc']]
#queries['query_adhoc'] = [expand_query(wiki_model, el) for el in queries['query_adhoc']]
#queries['query_adhoc'] = [porter_stemmer(el) for el in queries['query_adhoc']]

queries['query_desc'] = [el.lower() for el in queries['query_desc']]
queries['query_desc'] = [remove_punctuation(el) for el in queries['query_desc']]
#queries['query_desc'] = [remove_stopwords(el) for el in queries['query_desc']]
#queries['query_desc'] = [porter_stemmer(el) for el in queries['query_desc']]

queries['query_nar'] = [el.lower() for el in queries['query_nar']]
queries['query_nar'] = [remove_punctuation(el) for el in queries['query_nar']]
#queries['query_nar'] = [remove_stopwords(el) for el in queries['query_nar']]
#queries['query_nar'] = [porter_stemmer(el) for el in queries['query_nar']]

queries.head()


,index,qid,query,query_adhoc,query_desc,query_nar
0,0,1,NaN,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,2,3,NaN,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,3,4,NaN,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,4,5,NaN,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


,index,qid,query,query_adhoc,query_desc,query_nar
0,0,1,NaN,coronavirus origin,what is the origin of covid19,seeking range of information about the sarscov...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the sarscov...
2,2,3,NaN,coronavirus immunity,will sarscov2 infected people develop immunity...,seeking studies of immunity developed due to i...
3,3,4,NaN,how do people die from the coronavirus,what causes death from covid19,studies looking at mechanisms of death from co...
4,4,5,NaN,animal models of covid19,what drugs have been active against sarscov or...,papers that describe the results of testing d...


In [ ]:
queries_adhoc = queries[['qid', 'query_adhoc']]
queries_adhoc.rename(columns = {"query_adhoc" : "query"}, inplace=True)

queries_desc = queries[['qid', 'query_desc']]
queries_desc.rename(columns = {"query_desc" : "query"}, inplace=True)

queries_narrative = queries[['qid', 'query_nar']]
queries_narrative.rename(columns = {"query_nar" : "query"}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:

from pyterrier.measures import *
import onir_pt

vbert = onir_pt.reranker.from_checkpoint('https://macavaney.us/scibert-medmarco.tar.gz', text_field='title', expected_md5="854966d0b61543ffffa44cea627ab63b")

tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF") % 100
pipeline_tfidf = tfidf >> pt.text.get_text(index, 'title') >> vbert
pipeline_tfidf_rewriting = tfidf >> pt.rewrite.RM3(index) >> pt.rewrite.linear(0.60, 0.40, format="terrierql") >>tfidf #>> pt.text.get_text(index, 'title') >> vbert
bm25 = pt.BatchRetrieve(index, wmodel="BM25") % 100
pipeline_bm25 = bm25 >> pt.text.get_text(index, 'title') >> vbert
pipeline_bm25_rewriting = bm25 >> pt.rewrite.RM3(index) >> pt.rewrite.linear(0.60, 0.40, format="terrierql") >>bm25 #>> pt.text.get_text(index, 'title') >> vbert
dir = pt.BatchRetrieve(index, wmodel="DirichletLM") % 100
pipeline_dir = dir >> pt.text.get_text(index, 'title') >> vbert
pipeline_dir_rewriting = dir >> pt.rewrite.RM3(index) >> pt.rewrite.linear(0.60, 0.40, format="terrierql") >>dir #>> pt.text.get_text(index, 'title') >> vbert
dph = pt.BatchRetrieve(index, wmodel="DPH") % 100
pipeline_dph = dph >> pt.text.get_text(index, 'title') >> vbert
pipeline_dph_rewriting = dph >> pt.rewrite.RM3(index) >> pt.rewrite.linear(0.60, 0.40, format="terrierql") >>dph #>> pt.text.get_text(index, 'title') >> vbert

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph, pipeline_tfidf_rewriting, pipeline_bm25_rewriting, pipeline_dir_rewriting, pipeline_dph_rewriting],
    queries_adhoc,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT", "tf-idf + linear", "bm25 + linear", "DirichletLM + linear", "DPH + linear"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
#results.to_csv("first.csv", index = False)

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph, pipeline_tfidf_rewriting, pipeline_bm25_rewriting, pipeline_dir_rewriting, pipeline_dph_rewriting],
    queries_desc,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT", "tf-idf + linear", "bm25 + linear", "DirichletLM + linear", "DPH + linear"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
#results.to_csv("second.csv", index = False)

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph, pipeline_tfidf_rewriting, pipeline_bm25_rewriting, pipeline_dir_rewriting, pipeline_dph_rewriting],
    queries_narrative,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT", "tf-idf + linear", "bm25 + linear", "DirichletLM + linear", "DPH + linear"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
#results.to_csv("third.csv", index = False)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
config file not found: config


[2022-01-18 17:10:06,205][onir.util.download][DEBUG] downloaded https://macavaney.us/scibert-medmarco.tar.gz [9.35s] [499M] [104MB/s] [md5 hash verified]


[2022-01-18 17:10:23,490][onir.util.download][DEBUG] downloaded https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar [12.69s] [411M] [35.8MB/s] [md5 hash verified]


extracting: 411MB [1.14s, 362MB/s]
extracting: 821MB [8.31s, 98.7MB/s]


[2022-01-18 17:10:59,736][onir_pt][DEBUG] using GPU (deterministic)
[2022-01-18 17:11:10,613][onir_pt][DEBUG] [starting] batches


...

batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-01-18 17:17:25,277][onir_pt][DEBUG] [finished] batches: [25.95s] [1250it] [48.16it/s]
[2022-01-18 17:17:28,763][onir_pt][DEBUG] using GPU (deterministic)
[2022-01-18 17:17:28,767][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-01-18 17:17:54,574][onir_pt][DEBUG] [finished] batches: [25.81s] [1250it] [48.44it/s]


,name,AP@1000,P@5,P@10,map,ndcg,R@5,R@10,RR
0,tf-idf,0.032910,0.440,0.426,0.032910,0.097529,0.004798,0.009124,0.549424
1,bm25,0.033340,0.416,0.420,0.033340,0.098223,0.004634,0.009112,0.565794
2,DirichletLM,0.020440,0.292,0.292,0.020440,0.071928,0.002958,0.005841,0.450707
3,DPH,0.032167,0.436,0.424,0.032167,0.095763,0.004694,0.009409,0.557333
4,tf-idf + BERT,0.043664,0.648,0.590,0.043664,0.111942,0.007411,0.013555,0.756703
5,bm25 + BERT,0.044010,0.644,0.598,0.044010,0.112457,0.007300,0.013630,0.757527
6,DirichletLM + BERT,0.030813,0.608,0.546,0.030813,0.088868,0.006851,0.012115,0.725293
7,DPH + BERT,0.042501,0.680,0.608,0.042501,0.110917,0.007816,0.013772,0.790006
8,tf-idf + linear,0.034703,0.444,0.418,0.034703,0.099850,0.004933,0.009038,0.530269
9,bm25 + linear,0.035167,0.428,0.422,0.035167,0.100458,0.004816,0.009280,0.537100
